# Setup

In [ ]:
import matplotlib.pyplot as plt

from helium import helium
from helium.common import GenerationConfig
from helium.frontend.programs.examples import MCQAProgram
from helium.runtime import HeliumServerConfig
from helium.runtime.protocol import HeliumRequestConfig

# Use local Helium server
server_config = HeliumServerConfig(is_local=True, llm_service_configs=[])

In [ ]:
# Get a Helium instance without starting the server
server = helium.get_instance(config=server_config)

# Graph optimization

## Create a program agent

In [ ]:
generation_config = GenerationConfig.from_env(temperature=1.0)
mcqa_program = MCQAProgram(
    num_agents=5,
    method="cot",
    generation_config=generation_config,
    server_config=server_config,
)

In [ ]:
# This is a composite agent embodying the compute graphs of individual agents
agent = mcqa_program.create_program_agent(input_name="user_inputs")

## Original compute graph

In [ ]:
plt.figure(figsize=(10, 10))
agent.graph.visualize()
print("The graph contains", agent.graph.node_count, "nodes.")

# Optimized compute graph

In [ ]:
# 1. Compile the agent's compute graph
agent.compile(user_inputs=["test"])

# 2. Optimize the compiled graph
query = {"graph": agent.get_and_reset_compiled_graph()}
optimized, _ = server.optimizer.optimize(query, HeliumRequestConfig())

# 3. Plot the optimized graph
plt.figure(figsize=(10, 10))
graph = optimized.graph
graph.visualize()
print("The graph contains", graph.node_count, "nodes.")